## labor_project_pyspark_datasets

(need java 8 for spark to work. 7zip also recommended to unzip DEA dataset)

In [1]:
import pyspark
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('pillsdata').getOrCreate()

In [4]:
spark

https://seer.cancer.gov/popdata/download.html

In [5]:
pop_data = spark.read.text('D:/opioids/us.1969_2020.19ages.adjusted.txt')

https://seer.cancer.gov/popdata/popdic.html

In [6]:
from pyspark.sql import functions as F

pop_data = pop_data.withColumn(
    "year",
    F.substring(F.col("value"), 1, 4)
)

pop_data = pop_data.withColumn(
    "state",
    F.substring(F.col("value"), 5, 2)
)

pop_data = pop_data.withColumn(
    "state_fip",
    F.substring(F.col("value"), 7, 2)
)

pop_data = pop_data.withColumn(
    "county_fip",
    F.substring(F.col("value"), 9, 3)
)

pop_data = pop_data.withColumn(
    "registry",
    F.substring(F.col("value"), 12, 2)
)
pop_data = pop_data.withColumn(
    "race",
    F.substring(F.col("value"), 14,1)
)
pop_data = pop_data.withColumn(
    "origin",
    F.substring(F.col("value"), 15, 1)
)

pop_data = pop_data.withColumn(
    "sex",
    F.substring(F.col("value"), 16, 1)
)

pop_data = pop_data.withColumn(
    "age",
    F.substring(F.col("value"), 17, 2)
)

pop_data = pop_data.withColumn(
    "population",
    F.substring(F.col("value"), 19, 8)
)

pop_data = pop_data.drop('value')

+----+-----+---------+----------+--------+----+------+---+---+----------+
|year|state|state_fip|county_fip|registry|race|origin|sex|age|population|
+----+-----+---------+----------+--------+----+------+---+---+----------+
|1969|   AL|       01|       001|      99|   1|     9|  1| 00|  00000159|
|1969|   AL|       01|       001|      99|   1|     9|  1| 01|  00000657|
|1969|   AL|       01|       001|      99|   1|     9|  1| 02|  00001137|
|1969|   AL|       01|       001|      99|   1|     9|  1| 03|  00000956|
|1969|   AL|       01|       001|      99|   1|     9|  1| 04|  00000721|
|1969|   AL|       01|       001|      99|   1|     9|  1| 05|  00000424|
|1969|   AL|       01|       001|      99|   1|     9|  1| 06|  00000585|
|1969|   AL|       01|       001|      99|   1|     9|  1| 07|  00000637|
|1969|   AL|       01|       001|      99|   1|     9|  1| 08|  00000607|
|1969|   AL|       01|       001|      99|   1|     9|  1| 09|  00000523|
|1969|   AL|       01|       001|     

In [7]:
from pyspark.sql.types import IntegerType
pop_data = pop_data.withColumn("population",pop_data.population.cast(IntegerType()))
pop_data = pop_data.withColumn("year",pop_data.year.cast(IntegerType()))

In [8]:
pop_data = pop_data.filter(pop_data.year>1997)

In [9]:
pop_data = pop_data.groupBy('year','state','state_fip','county_fip','registry').sum('population')

In [10]:
pop_data = pop_data.withColumnRenamed("sum(population)", "population")

https://www.washingtonpost.com/national/2019/07/18/how-download-use-dea-pain-pills-database/

In [11]:
DEA_data = spark.read.csv('D:/opioids/arcos_all_washpost/arcos_all_washpost.tsv', sep='\t',header=True)

In [12]:
DEA_data.show()

+---------------+----------------+--------------------+---------------------+-----------------+-----------------+-------------+--------------+------------+---------------+------------+-------------------+--------------------+--------------------+--------------------+---------------+----------+-----------+---------+------------+----------------+---------+-----------+-----------+--------+----+----------------+-------------+-------------+--------+----------------+------------------+-----------+--------------+--------------------+--------------------+-------+---------------------+---------------------+--------------------+--------------------+-------+
|REPORTER_DEA_NO|REPORTER_BUS_ACT|       REPORTER_NAME|REPORTER_ADDL_CO_INFO|REPORTER_ADDRESS1|REPORTER_ADDRESS2|REPORTER_CITY|REPORTER_STATE|REPORTER_ZIP|REPORTER_COUNTY|BUYER_DEA_NO|      BUYER_BUS_ACT|          BUYER_NAME|  BUYER_ADDL_CO_INFO|      BUYER_ADDRESS1| BUYER_ADDRESS2|BUYER_CITY|BUYER_STATE|BUYER_ZIP|BUYER_COUNTY|TRANSACTION_COD

In [13]:
DEA_data = DEA_data.select(['BUYER_STATE','BUYER_COUNTY','DRUG_NAME','QUANTITY','TRANSACTION_DATE'])
DEA_data = DEA_data.filter((DEA_data.DRUG_NAME == 'HYDROCODONE') & (DEA_data.DRUG_NAME == 'OXYCODONE'))

In [14]:
DEA_data = DEA_data.withColumn(
    "year",
    F.substring(F.col("TRANSACTION_DATE"), 5, 4)
)

DEA_data = DEA_data.withColumn(
    "month",
    F.substring(F.col("TRANSACTION_DATE"), 1, 2)
)

In [71]:
DEA_data = DEA_data.drop('TRANSACTION_DATE')

In [17]:
DEA_data = DEA_data.withColumn("QUANTITY",DEA_data.QUANTITY.cast(IntegerType()))
DEA_data = DEA_data.groupBy('BUYER_STATE','BUYER_COUNTY','DRUG_NAME','year','month').sum('QUANTITY')
DEA_data = DEA_data.withColumnRenamed("sum(DEA_data)", "quantity")

In [24]:
from pyspark.sql.functions import *
DEA_data = DEA_data.withColumn("year",DEA_data.year.cast(IntegerType()))
maxyear = DEA_data.select(max(col('year')).alias('maxyear')).first().maxyear
minyear = DEA_data.select(min(col('year')).alias('minyear')).first().minyear

In [ ]:
pop_data = pop_data.filter((pop_data.year<=maxyear) & (pop_data.year>=minyear))

In [ ]:
merged_data = DEA_data.join(pop_data , on=['year','BUYER_COUNTY'] , how = 'left')

In [ ]:
merged_data.show()

In [ ]:
merged_data = merged_data.withColumn('pills_per_capita', merged_data.QUANTITIES/merged_data.population)

In [ ]:
reduced_data = merged_data.select(['year','month','BUYER_STATE','BUYER_COUNTY','state_fip','county_fip','pills_per_capita'])
reduced_data = reduced_data.withColumnsRenamed({'BUYER_STATE': 'state_name', 'BUYER_COUNTY': 'county_name'})

In [ ]:
reduced_data.toPandas().to_csv('D:/opioids/reduced_opioid_data.csv')